<a href="https://colab.research.google.com/github/superishi/Cybersecurity-Flow-Anomaly-Detection/blob/main/WeekdayAnomalyClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Creating extra columns to be used for filtering
### time aggregated for every 3 hours, model focuses on 09:00 - 17:00 time period

In [3]:
def extract_day(time_string):
  day_string = time_string[:10]
  return day_string

In [4]:
def extract_time(time_string):
  time_string = time_string[-8:]
  return time_string

In [5]:
origData = pd.read_csv("/content/drive/MyDrive/DATA3001 Cyber/by3hours.csv", header = 0)
#print(origData)
origData['edit_date'] = origData['time'] 
origData['edit_date'] = origData['edit_date'].apply(extract_day)

origData['edit_time'] = origData['time']
origData['edit_time'] = origData['edit_time'].apply(extract_time)

origData = origData.drop(origData[origData.edit_date == "2021-08-28"].index )
origData = origData.drop(origData[origData.edit_date == "2021-08-29"].index )
origData = origData.drop(origData[origData.edit_date == "2021-09-04"].index )
origData = origData.drop(origData[origData.edit_date == "2021-09-05"].index )

remove_time_set = ["00:00:00", "03:00:00", "06:00:00", "18:00:00", "21:00:00"]

origData = origData.drop(origData[origData.edit_time.isin(remove_time_set)].index )

print(origData)

                      time   device_mac  ...   edit_date  edit_time
0      2021-08-23 15:00:00  21:fa:27:56  ...  2021-08-23   15:00:00
6      2021-08-24 09:00:00  21:fa:27:56  ...  2021-08-24   09:00:00
7      2021-08-24 12:00:00  21:fa:27:56  ...  2021-08-24   12:00:00
8      2021-08-24 15:00:00  21:fa:27:56  ...  2021-08-24   15:00:00
14     2021-08-25 09:00:00  21:fa:27:56  ...  2021-08-25   09:00:00
...                    ...          ...  ...         ...        ...
85166  2021-09-03 12:00:00  ee:dd:4f:cd  ...  2021-09-03   12:00:00
85167  2021-09-03 15:00:00  ee:dd:4f:cd  ...  2021-09-03   15:00:00
85189  2021-09-06 09:00:00  ee:dd:4f:cd  ...  2021-09-06   09:00:00
85190  2021-09-06 12:00:00  ee:dd:4f:cd  ...  2021-09-06   12:00:00
85191  2021-09-06 15:00:00  ee:dd:4f:cd  ...  2021-09-06   15:00:00

[23369 rows x 8 columns]


In [7]:
excluded_devices = ["bd:1c:dc:fb", "ac:f2:14:98", "ac:f4:65:4a", "bd:0d:af:4f"]

origData = origData.drop(origData[origData.device_mac.isin(excluded_devices)].index )
origData

,time,device_mac,v0,v1,v2,v3,edit_date,edit_time
0,2021-08-23 15:00:00,21:fa:27:56,0.003341,0.942904,-0.278402,-0.181698,2021-08-23,15:00:00
6,2021-08-24 09:00:00,21:fa:27:56,0.003341,0.942907,-0.278401,-0.181683,2021-08-24,09:00:00
7,2021-08-24 12:00:00,21:fa:27:56,0.003341,0.942907,-0.278401,-0.181683,2021-08-24,12:00:00
8,2021-08-24 15:00:00,21:fa:27:56,0.003341,0.942905,-0.278402,-0.181693,2021-08-24,15:00:00
14,2021-08-25 09:00:00,21:fa:27:56,0.003344,0.943003,-0.278053,-0.181722,2021-08-25,09:00:00
...,...,...,...,...,...,...,...,...
85166,2021-09-03 12:00:00,ee:dd:4f:cd,0.999938,-0.004904,-0.004676,-0.000309,2021-09-03,12:00:00
85167,2021-09-03 15:00:00,ee:dd:4f:cd,0.999938,-0.004904,-0.004676,-0.000309,2021-09-03,15:00:00
85189,2021-09-06 09:00:00,ee:dd:4f:cd,0.999952,-0.003872,-0.001447,-0.000458,2021-09-06,09:00:00
85190,2021-09-06 12:00:00,ee:dd:4f:cd,0.999936,-0.004898,-0.005163,-0.000095,2021-09-06,12:00:00


In [31]:
deviceLabels = pd.read_csv("/content/drive/MyDrive/DATA3001 Cyber/WeekdayDeviceWithClusterLabels3Hours.csv", header = None, names = ['device_mac', 'cluster_label'])
#deviceLabels = deviceLabels.set_index(0)
deviceLabels

,device_mac,cluster_label
0,21:fa:27:56,2
1,21:fa:27:82,2
2,21:fa:29:50,2
3,21:fa:2d:cf,0
4,21:fa:2e:47,2
...,...,...
748,ee:dd:4e:5f,0
749,ee:dd:4e:60,3
750,ee:dd:4f:4c,0
751,ee:dd:4f:57,0


### Left joining tables to add cluster label column

In [32]:
#deviceLabels = deviceLabels.set_index('0')
origData = origData.merge(deviceLabels, how = 'left', on = 'device_mac')
origData.iloc[100]

time               2021-08-26 09:00:00
device_mac                 21:fa:2d:cf
v0                                   0
v1                                   0
v2                                   0
v3                                   0
edit_date                   2021-08-26
edit_time                     09:00:00
cluster_label_x                      0
cluster_label_y                      0
cluster_label_x                      0
cluster_label_y                      0
cluster_label                        0
Name: 100, dtype: object

In [36]:
cluster_data = origData.drop(['time', 'device_mac', 'edit_date', 'edit_time', 'cluster_label_x', 'cluster_label_y'], axis = 1)
cluster_data

,v0,v1,v2,v3,cluster_label
0,0.003341,0.942904,-0.278402,-0.181698,2
1,0.003341,0.942907,-0.278401,-0.181683,2
2,0.003341,0.942907,-0.278401,-0.181683,2
3,0.003341,0.942905,-0.278402,-0.181693,2
4,0.003344,0.943003,-0.278053,-0.181722,2
...,...,...,...,...,...
23338,0.999938,-0.004904,-0.004676,-0.000309,1
23339,0.999938,-0.004904,-0.004676,-0.000309,1
23340,0.999952,-0.003872,-0.001447,-0.000458,1
23341,0.999936,-0.004898,-0.005163,-0.000095,1


In [37]:
cluster_data_np = cluster_data.to_numpy()
print(cluster_data_np.shape)
x_data_points = cluster_data_np[:,:-1]
print(x_data_points)
y_labels = cluster_data_np[:,-1]
print(y_labels)

(23343, 5)
[[ 3.34081050e-03  9.42904424e-01 -2.78401617e-01 -1.81697554e-01]
 [ 3.34083192e-03  9.42907371e-01 -2.78401398e-01 -1.81682670e-01]
 [ 3.34083145e-03  9.42907304e-01 -2.78401393e-01 -1.81682912e-01]
 ...
 [ 9.99952437e-01 -3.87153356e-03 -1.44708658e-03 -4.58177364e-04]
 [ 9.99935824e-01 -4.89825150e-03 -5.16344308e-03 -9.45717623e-05]
 [ 9.99938694e-01 -4.79479178e-03 -4.67775496e-03 -1.81121297e-04]]
[2. 2. 2. ... 1. 1. 1.]


## Using K-Neighbours Classifier with imported clusters
### data points are the 3 hour-aggregated flows

In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [39]:
 X_train, X_test, y_train, y_test = train_test_split(x_data_points, y_labels, test_size=0.2, random_state=420)
 print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
 print(X_test)

(18674, 4) (4669, 4) (18674,) (4669,)
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 3.37879895e-03  9.43958852e-01 -2.74653490e-01 -1.81927578e-01]
 [ 9.85921919e-03  3.01181154e-01  9.52693635e-01 -3.81706249e-02]
 ...
 [ 9.92384471e-03  3.21656466e-01  9.45748289e-01 -4.37742705e-02]
 [ 9.99907875e-01 -5.86452810e-03 -8.50209435e-03  8.50839712e-06]
 [ 3.35954969e-03  9.43428849e-01 -2.76556046e-01 -1.81794598e-01]]


In [40]:
def num_neighbors_param_test():
  for i in range(1, 50):
    KNmodel = KNeighborsClassifier(n_neighbors = i)
    KNmodel.fit(X_train, y_train)
    results = KNmodel.score(X_test, y_test)
    print(i, " : ", results)

In [43]:
num_neighbors_param_test()

1  :  0.6277575497965303
2  :  0.6373955879203256
3  :  0.6630970229171129
4  :  0.6626686656671664
5  :  0.6699507389162561
6  :  0.679160419790105
7  :  0.6667380595416578
8  :  0.6789462411651317
9  :  0.6806596701649176
10  :  0.6620261297922467
11  :  0.6772328121653459
12  :  0.67551938316556
13  :  0.6761619190404797
14  :  0.6748768472906403
15  :  0.6774469907903191
16  :  0.6714499892910688
17  :  0.6731634182908546
18  :  0.6720925251659884
19  :  0.6731634182908546
20  :  0.6718783465410152
21  :  0.6718783465410152
22  :  0.6697365602912829
23  :  0.6712358106660955
24  :  0.6708074534161491
25  :  0.6705932747911758
26  :  0.6710216320411223
27  :  0.6697365602912829
28  :  0.6690940244163632
29  :  0.6697365602912829
30  :  0.6695223816663097
31  :  0.6699507389162561
32  :  0.6686656671664168
33  :  0.6695223816663097
34  :  0.6680231312914972
35  :  0.66887984579139
36  :  0.6667380595416578
37  :  0.666952238166631
38  :  0.6665238809166845
39  :  0.6665238809166845
4

### 9 neighbours chosen based on hyperparameter test

In [42]:
KNmodel = KNeighborsClassifier(n_neighbors = 9)
KNmodel.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

In [44]:
results = KNmodel.score(X_train, y_train)
results

0.7246438899003963

In [45]:
res1 = KNmodel.predict(X_train)
res1

array([0., 0., 2., ..., 2., 3., 2.])

In [46]:
res2 = KNmodel.predict_proba(X_test)
res2.shape

(4669, 4)

### Get probability that data point belongs to its actual cluster according to model

### if probability is less than 10%, flag as potential anomaly

In [49]:
prob_of_correct = np.ones((4669,1))

for i in range(4669):
  correct_cluster = int(y_test[i])
  prob_of_correct[i] = res2[i, correct_cluster]

In [52]:
print(prob_of_correct.shape[0])
total = prob_of_correct.shape[0]
print(prob_of_correct[prob_of_correct > 0.1].shape[0])
condition = prob_of_correct[prob_of_correct > 0.10].shape[0]

print(condition/total)

4669
4358
0.9333904476333262
